# Collect "Exchange" data
爬取 2018/01/01 至 2018/05/16的資料 (共87個交易日)

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import time

**建立欲爬取日期的清單：**

In [19]:
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2018, 5, 16)
step = datetime.timedelta(days = 1)

#創造日期清單
date_string = []
while start <= end:
    date_string.append(start.strftime('%Y-%m-%d'))
    start += step

#定義函式將清單的datetime object轉成數字，才能pass進爬蟲裡
def date_string_to_number(data):
    temp = pd.DataFrame(str.split(data, "-")).iloc[:, 0].apply(int)
    result = temp[0] * 10000 + temp[1] * 100 + temp[2]
    return(result)

#進行日期格式轉換
date_string = pd.DataFrame(date_string).iloc[:, 0].apply(date_string_to_number)

In [ ]:
from Crawler import * #使用原先未使用OOP的版本(沒有包含整理功能因此以下需要自行整理)

## 爬資料

**爬三大法人買賣超**

In [99]:
# main_force = pd.DataFrame([])
# for i in date_string:
#     temp = get_main_force("ALLBUT0999", i)
#     main_force = pd.concat([main_force, temp], )
#     print("Date : {} finished.".format(i))
#     time.sleep(10)

# main_force.to_excel("2018_三大法人買賣超.xlsx")

**爬融資融券**

In [22]:
# margin_purchase_short_sell = pd.DataFrame([])
# for i in date_string:
#     temp = get_margin_purchase_short_sell("ALL", i)
#     margin_purchase_short_sell = pd.concat([margin_purchase_short_sell, temp], )
#     print("Date : {} finished.".format(i))
#     time.sleep(5)
    
# margin_purchase_short_sell.to_excel("2018_融資融券.xlsx")

**爬借券**

*(此部分爬下來前幾欄為融券後幾欄才為借券，因此後面裡只要選取借券部分)*

In [43]:
# borrow = pd.DataFrame([])
# for i in date_string:
#     temp = get_borrow("ALL", i)
#     borrow = pd.concat([borrow, temp], )
#     print("Date : {} finished.".format(i))
#     time.sleep(5)

# borrow.to_excel("2018_借券.xlsx")

## 整理資料

**三大法人買賣超**

In [214]:
main_force = pd.read_excel("2018_三大法人買賣超.xlsx")
main_force = main_force[main_force.證券代號.apply(lambda x : len(x) == 4)]
main_force = main_force[["日期", '證券代號', '證券名稱', '外陸資買賣超股數(不含外資自營商)', '投信買賣超股數']]
main_force.columns = ["日期", '股票代號', '股票名稱', '外陸資買賣超股數(不含外資自營商)', '投信買賣超股數']
main_force.iloc[:, [3, 4]] = main_force.iloc[:, [3, 4]].applymap(lambda x : float(x.replace(",", ""))) / 1000
main_force.head()

,日期,股票代號,股票名稱,外陸資買賣超股數(不含外資自營商),投信買賣超股數
1,2018-01-02,2603,長榮,7397.440,4190.0
3,2018-01-02,2330,台積電,9607.424,514.0
4,2018-01-02,2014,中鴻,755.000,864.0
5,2018-01-02,3481,群創,6269.728,0.0
6,2018-01-02,6120,達運,5271.901,0.0


**融資融券**

In [215]:
margin_purchase_short_sell = pd.read_excel("2018_融資融券.xlsx")
margin_purchase_short_sell = margin_purchase_short_sell[margin_purchase_short_sell.股票代號.apply(lambda x : len(x) == 4)]
margin_purchase_short_sell.iloc[:, [5, 6, 11, 12]] = margin_purchase_short_sell.iloc[:, [5, 6, 11, 12]].applymap(lambda x : float(x.replace(",", "")))
margin_purchase_short_sell["融資張數"] = margin_purchase_short_sell.iloc[:, 6] - margin_purchase_short_sell.iloc[:, 5]
margin_purchase_short_sell["融券張數"] = margin_purchase_short_sell.iloc[:, 12] - margin_purchase_short_sell.iloc[:, 11]
margin_purchase_short_sell = margin_purchase_short_sell[["日期", '股票代號', '股票名稱', "融資張數", "融券張數"]]

margin_purchase_short_sell.head()

,日期,股票代號,股票名稱,融資張數,融券張數
0,2018-01-02,0050,元大台灣50,-170.0,18.0
1,2018-01-02,0051,元大中型100,0.0,0.0
2,2018-01-02,0052,富邦科技,0.0,0.0
3,2018-01-02,0053,元大電子,0.0,0.0
4,2018-01-02,0054,元大台商50,0.0,0.0


**借券**

In [216]:
borrow = pd.read_excel("2018_借券.xlsx")
borrow = borrow[borrow.股票代號.apply(lambda x : len(str(x)) == 4)]
borrow.iloc[:, [8, 12]] = borrow.iloc[:, [8, 12]].applymap(lambda x : float(x.replace(",", "")))
borrow["借券張數"] = (borrow.iloc[:, 12] - borrow.iloc[:, 8]) / 1000
borrow = borrow[["日期", '股票代號', '股票名稱', "借券張數"]]

borrow.head()

,日期,股票代號,股票名稱,借券張數
0,2018-01-02,0050,元大台灣50,-112.0
1,2018-01-02,0051,元大中型100,0.0
2,2018-01-02,0052,富邦科技,0.0
3,2018-01-02,0053,元大電子,0.0
4,2018-01-02,0054,元大台商50,0.0


**淨流通在外張數**

*(此資料依據 2018/5/15 日TEJ資料)*

In [477]:
# outstanding_stock = pd.read_excel("2018_董監持股_流通張數.xlsx")
# outstanding_stock["淨流通在外張數"] = outstanding_stock["流通在外張數"] - outstanding_stock["董監持張數"]
# outstanding_stock.股票名稱 = outstanding_stock.股票名稱.apply(lambda x : x.split(" ")[1])
# outstanding_stock = outstanding_stock[["股票代號", "股票名稱", "淨流通在外張數"]]
# outstanding_stock.dropna(inplace = True)
# outstanding_stock.to_excel("2018_董監持股_流通張數.xlsx")

outstanding_stock = pd.read_excel("2018_董監持股_流通張數.xlsx")
outstanding_stock.head()

,股票代號,股票名稱,淨流通在外張數
0,1101,台泥,3631883
1,1102,亞泥,2385778
2,1103,嘉泥,636994
3,1104,環泥,495284
4,1108,幸福,367418


**合併全部**

In [274]:
result = main_force.merge(margin_purchase_short_sell, on = ["日期", "股票代號"], how = "outer").merge(borrow, on = ["日期", "股票代號"], how = "outer")
result = result[['日期', '股票代號', '股票名稱', '外陸資買賣超股數(不含外資自營商)', '投信買賣超股數', '借券張數', '融資張數', '融券張數']]
result.to_excel("2018_temp_result.xlsx")